# DataFrames: Reading in messy data
     
In the [01-data-access](./01-data-access.ipynb) example we show how Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  One key difference, when using Dask Dataframes is that instead of opening a single file with a function like [pandas.read_csv](https://docs.dask.org/en/latest/generated/dask.dataframe.read_csv.html), we typically open many files at once with [dask.dataframe.read_csv](https://docs.dask.org/en/latest/dataframe-api.html#dask.dataframe.read_csv). This enables us to treat a collection of files as a single dataset. Most of the time this works really well. But real data is messy and in this notebook we will explore a more advanced technique to bring messy datasets into a dask dataframe.

## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [1]:
from dask.distributed import Client
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 10
Total threads: 80,Total memory: 0.98 TiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:44325,Workers: 10
Dashboard: http://127.0.0.1:8787/status,Total threads: 80
Started: Just now,Total memory: 0.98 TiB
Comm: tcp://127.0.0.1:41159,Total threads: 8
Dashboard: http://127.0.0.1:37157/status,Memory: 100.78 GiB
Nanny: tcp://127.0.0.1:39153,


## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [2]:
import dask
df = dask.datasets.timeseries()
df

,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In [3]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name, index=False);

## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [4]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [5]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

,id,name,x,y
npartitions=30,,,,
,int64,object,float64,float64
,...,...,...,...
...,...,...,...,...
,...,...,...,...
,...,...,...,...


In [6]:
df.head()

,id,name,x,y
0,975,Quinn,-0.257187,0.536141
1,999,Wendy,0.271711,-0.995159
2,982,Ursula,0.451689,-0.128281
3,1029,Bob,0.044694,-0.226989
4,973,Michael,0.557343,0.568602


Let's look at some statistics on the data

In [7]:
df.describe().compute()

,id,x,y
count,2.592000e+06,2.592000e+06,2.592000e+06
mean,9.999938e+02,7.299987e-04,-8.035330e-04
std,3.163807e+01,5.774100e-01,5.773645e-01
min,8.420000e+02,-9.999999e-01,-9.999999e-01
25%,9.790000e+02,-4.933656e-01,-4.967885e-01
50%,1.000000e+03,1.216692e-02,6.484002e-03
75%,1.022000e+03,5.070924e-01,5.044880e-01
max,1.159000e+03,9.999984e-01,9.999995e-01


# Make some messy data

Now this works great, and in most cases dd.read_csv or dd.read_parquet etc are the preferred way to read in large collections of data files into a dask dataframe, but real world data is often very messy and some files may be broken or badly formatted. To simulate this we are going to create some fake messy data by tweaking our example csv files. For the file `data/2000-01-05.csv` we will replace with no data and for the file `data/2000-01-07.csv` we will remove the `y` column 

In [8]:
# corrupt the data in data/2000-01-05.csv
with open('data/2000-01-05.csv', 'w') as f:
    f.write('')

In [9]:
# remove y column from data/2000-01-07.csv
import pandas as pd
df = pd.read_csv('data/2000-01-07.csv')
del df['y']
df.to_csv('data/2000-01-07.csv', index=False)

In [10]:
!head data/2000-01-05.csv

In [11]:
!head data/2000-01-07.csv

id,name,x
987,Ray,0.6802690177110982
1038,Laura,-0.2190117162455491
1025,Sarah,0.189413659167172
1033,Michael,-0.6606807667925605
1008,Bob,-0.1844813523965758
1007,Xavier,0.2026711461430772
981,Hannah,-0.920595047509162
1032,Quinn,-0.9255521582027232
966,Ursula,0.5808226279638828


# Reading the messy data

Let's try reading in the collection of files again

In [12]:
df = dd.read_csv('data/2000-*-*.csv')

In [13]:
df.head()

,id,name,x,y
0,975,Quinn,-0.257187,0.536141
1,999,Wendy,0.271711,-0.995159
2,982,Ursula,0.451689,-0.128281
3,1029,Bob,0.044694,-0.226989
4,973,Michael,0.557343,0.568602


Ok this looks like it worked, let us calculate the dataset statistics again

In [14]:
try:
    df.describe().compute()
except ValueError as e:
    print(str(e))

distributed.worker - WARNING - Compute Failed
Function:  execute_task
args:      ((subgraph_callable-998cfd86-c8b8-471a-97fd-d9c80d91e7bc, [(<function read_block_from_file at 0x7fa6d73cb1f0>, <OpenFile '/datasets/charlesb/dev/dask/dask-examples/dataframes/data/2000-01-07.csv'>, 0, 2611066, b'\n'), None, True, True]))
kwargs:    {}
Exception: 'ValueError("The columns in the computed data do not match the columns in the provided metadata\\n  Extra:   []\\n  Missing: [\'y\']")'



ValueError: The columns in the computed data do not match the columns in the provided metadata
  Extra:   []
  Missing: ['y']

So what happened? 

When creating a dask dataframe from a collection of files, dd.read_csv samples the first few files in the dataset to determine the datatypes and columns available. Since it has not opened all the files it does not now if some of them are corrupt. Hence, `df.head()` works since it is only looking at the first file. `df.describe.compute()` fails because of the corrupt data in `data/2000-01-05.csv`

# Building a delayed reader

To get around this problem we are going to use a more advanced technique to build our dask dataframe. This method can also be used any time some custom logic is required when reading each file. Essentially, we are going to build a function that uses  pandas and some error checking and returns a pandas dataframe. If we find a bad data file we will either find a way to fix/clean the data or we will return and empty pandas dataframe with the same structure as the good data.

In [15]:
import numpy as np
import io

def read_data(filename):
    
    # for this to work we need to explicitly set the datatypes of our pandas dataframe 
    dtypes = {'id': int, 'name': str, 'x': float, 'y': float}
    try:
        # try reading in the data with pandas 
        df = pd.read_csv(filename, dtype=dtypes)
    except:
        # if this fails create an empty pandas dataframe with the same dtypes as the good data
        df = pd.read_csv(io.StringIO(''), names=dtypes.keys(), dtype=dtypes)
    
    # for the case with the missing column, add a column of data with NaN's
    if 'y' not in df.columns:
        df['y'] = np.NaN
        
    return df

Let's test this function on a good file and the two bad files

In [16]:
# test function on a normal file
read_data('data/2000-01-01.csv').head()

,id,name,x,y
0,975,Quinn,-0.257187,0.536141
1,999,Wendy,0.271711,-0.995159
2,982,Ursula,0.451689,-0.128281
3,1029,Bob,0.044694,-0.226989
4,973,Michael,0.557343,0.568602


In [17]:
# test function on the empty file
read_data('data/2000-01-05.csv').head()

,id,name,x,y


In [18]:
# test function on the file missing the y column
read_data('data/2000-01-07.csv').head()

,id,name,x,y
0,987,Ray,0.680269,NaN
1,1038,Laura,-0.219012,NaN
2,1025,Sarah,0.189414,NaN
3,1033,Michael,-0.660681,NaN
4,1008,Bob,-0.184481,NaN


# Assembling the dask dataframe

First we take our `read_data` function and convert it to a dask delayed function

In [19]:
from dask import delayed
read_data = delayed(read_data)

Let us look at what the function does now

In [20]:
df = read_data('data/2000-01-01.csv')
df

Delayed('read_data-cfe798db-fc89-460f-b5b0-5cc87ee42698')

It creates a delayed object, to actually run read the file we need to run `.compute()`

In [21]:
df.compute()

,id,name,x,y
0,975,Quinn,-0.257187,0.536141
1,999,Wendy,0.271711,-0.995159
2,982,Ursula,0.451689,-0.128281
3,1029,Bob,0.044694,-0.226989
4,973,Michael,0.557343,0.568602
...,...,...,...,...
86395,1007,Frank,-0.621999,0.964328
86396,1062,Zelda,-0.666499,0.498459
86397,987,Quinn,0.246710,0.645429
86398,982,Wendy,-0.873152,-0.000764


Now let's build a list of all the available csv files

In [22]:
# loop over all the files
from glob import glob
files = glob('data/2000-*-*.csv')
files

['data/2000-01-01.csv',
 'data/2000-01-02.csv',
 'data/2000-01-11.csv',
 'data/2000-01-12.csv',
 'data/2000-01-13.csv',
 'data/2000-01-14.csv',
 'data/2000-01-15.csv',
 'data/2000-01-16.csv',
 'data/2000-01-17.csv',
 'data/2000-01-18.csv',
 'data/2000-01-19.csv',
 'data/2000-01-20.csv',
 'data/2000-01-03.csv',
 'data/2000-01-21.csv',
 'data/2000-01-22.csv',
 'data/2000-01-23.csv',
 'data/2000-01-24.csv',
 'data/2000-01-25.csv',
 'data/2000-01-26.csv',
 'data/2000-01-27.csv',
 'data/2000-01-28.csv',
 'data/2000-01-29.csv',
 'data/2000-01-30.csv',
 'data/2000-01-04.csv',
 'data/2000-01-06.csv',
 'data/2000-01-05.csv',
 'data/2000-01-07.csv',
 'data/2000-01-08.csv',
 'data/2000-01-09.csv',
 'data/2000-01-10.csv']

Now we run the delayed read_data function on each file in the list

In [23]:
df = [read_data(file) for file in files]
df

[Delayed('read_data-0c6661d5-b1e8-46e0-9936-11139db5f9ec'),
 Delayed('read_data-90168c95-d19e-4173-b884-1780c8416ea3'),
 Delayed('read_data-29dada5d-bbe3-496d-93bd-cf40ef514355'),
 Delayed('read_data-1409ec2e-9b0b-4fb8-a54d-14cb13227e69'),
 Delayed('read_data-b4d87e31-b624-4fb2-a562-236377794387'),
 Delayed('read_data-e523bd0d-a8dd-440d-ba90-7b3837cc93a1'),
 Delayed('read_data-c40afd41-b747-43f5-a5cb-0655f9bf77ca'),
 Delayed('read_data-a120e049-369c-400d-bcc4-76e44d90f764'),
 Delayed('read_data-944614d0-01a9-45ae-bc15-e128932237e1'),
 Delayed('read_data-5575f546-587f-4945-acfe-3b3f12221ce3'),
 Delayed('read_data-d825a356-5ccc-4c6c-8a5e-105118101be0'),
 Delayed('read_data-a6ab238f-44b3-4bb0-b96b-0d54eedc3e5b'),
 Delayed('read_data-f5d4d3d1-3ba5-42f1-8c10-2293e8d7b6bc'),
 Delayed('read_data-600f1a68-cb96-4243-9229-642f5db6e33c'),
 Delayed('read_data-158f3b27-77a5-43b7-8f1a-852a5e4ca51c'),
 Delayed('read_data-26b6fe5b-9ef1-4c72-a660-c42603b550c9'),
 Delayed('read_data-b5879da3-edb2-4d15-a

Then we use [dask.dataframe.from_delayed](https://docs.dask.org/en/latest/generated/dask.dataframe.from_delayed.html). This function creates a Dask DataFrame from a list of delayed objects as long as each delayed object returns a pandas dataframe. The structure of each individual dataframe returned must also be the same.

In [24]:
df = dd.from_delayed(df, meta={'id': int, 'name': str, 'x': float, 'y': float})
df

,id,name,x,y
npartitions=30,,,,
,int64,object,float64,float64
,...,...,...,...
...,...,...,...,...
,...,...,...,...
,...,...,...,...


Note: we provided the dtypes in the `meta` keyword to explicitly tell Dask Dataframe what kind of dataframe to expect. If we did not do this Dask would infer this from the first delayed object which could be slow if it was a large csv file

## Now let's see if this works

In [25]:
df.head()

,id,name,x,y
0,975,Quinn,-0.257187,0.536141
1,999,Wendy,0.271711,-0.995159
2,982,Ursula,0.451689,-0.128281
3,1029,Bob,0.044694,-0.226989
4,973,Michael,0.557343,0.568602


In [26]:
df.describe().compute()

,id,x,y
count,2.505600e+06,2.505600e+06,2.419200e+06
mean,9.999930e+02,7.848738e-04,-8.131112e-04
std,3.163564e+01,5.773810e-01,5.773652e-01
min,8.420000e+02,-9.999999e-01,-9.999999e-01
25%,9.790000e+02,-4.933656e-01,-4.967885e-01
50%,1.000000e+03,1.216692e-02,6.484002e-03
75%,1.022000e+03,5.070924e-01,5.044880e-01
max,1.159000e+03,9.999984e-01,9.999995e-01


## Success!

To recap, in this example, we looked at an approach to create a Dask Dataframe from a collection of many data files. Typically you would use built-in functions like `dd.read_csv` or `dd.read_parquet` to do this. Sometimes, this is not possible because of messy/corrupted files in your dataset or some custom processing that might need to be done. 

In these cases, you can build a Dask DataFrame with the following steps.

1. Create a regular python function that reads the data, performs any transformations, error checking etc and always returns a  Pandas dataframe with the same structure
2. Convert this read function to a delayed object using the `dask.delayed` function
3. Call each file in your dataset with the delayed data reader and assemble the output as a list of delayed objects
4. Used `dd.from_delayed` to covert the list of delayed objects to a Dask Dataframe 

This same technique can be used in other situations as well. Another example might be data files that require using a specialized reader, or several transformations before they can be converted to a pandas dataframe.